In [6]:
import pandas as pd 
import collections
from datetime import timedelta
from dateutil.relativedelta import relativedelta
import numpy as np

import math
from datetime import datetime 
from datetime import datetime, timedelta


from powernad.API.Campaign import *
from powernad.API.RelKwdStat import *

import time

from time import sleep
from urllib.error import HTTPError

import urllib.request
from datetime import datetime, timedelta
import json

from tqdm.notebook import tqdm

In [7]:
toda = datetime.now()
time_month = toda - relativedelta(months=1)
time_month = time_month.strftime('%Y-%m-%d')
time_month= str(time_month)


yesterday = toda - relativedelta(days=1)
yesterday = yesterday.strftime('%Y-%m-%d')
yesterday = str(yesterday)



today = str(datetime.now().date())

In [8]:
dt_index = pd.date_range(start=time_month, end = yesterday)

date = pd.DataFrame(data=dt_index, columns=['날짜'])

In [9]:
BASE_URL = 'https://api.naver.com'
API_KEY = '0100000000aedd7e21ca248851339717a8c09555ed20e1e5cede79edfd948be51224d67f62'
SECRET_KEY = 'AQAAAACu3X4hyiSIUTOXF6jAlVXt0G36rz8arbVtCcBmmpCK0Q=='
CUSTOMER_ID = '1326555'

In [10]:
rel = RelKwdStat(BASE_URL, API_KEY, SECRET_KEY, CUSTOMER_ID)

def search_keyword(searchword):
    kwdDataList = rel.get_rel_kwd_stat_list(siteId=None, biztpId=None, hintKeywords=searchword, event=None, month=None, showDetail='1')
    kwd_result = (kwdDataList[0].relKeyword, #키워드
                 kwdDataList[0].monthlyPcQcCnt, #월간 검색수 (PC)
                 kwdDataList[0].monthlyMobileQcCnt, # 월간 검색수 (Mobile)
                 kwdDataList[0].monthlyPcQcCnt+kwdDataList[0].monthlyMobileQcCnt) # 월간 total 
    return(kwd_result[3])

client_id = "VwYdRR6MqJZJLGEOGqYr" 
client_secret = "9XtlZIiC59"

dictionary_yester={}
dictionary_now={}

In [38]:
# lis = ['검색하고자 하는 키워드 입력']
lis = ['위워크']

In [39]:
error=[]
for i in tqdm(lis):  
    sleep(1)
    try:
        if type(search_keyword(i)) !=str : 
            searchword = i.replace(" ","")
            a = search_keyword(searchword)


            url = "https://openapi.naver.com/v1/datalab/search"
            body = "{\"startDate\":\""+time_month+"\",\"endDate\":\""+today+"\",\"timeUnit\":\"date\",\"keywordGroups\":[{\"groupName\":\""+i+"\",\"keywords\":[\""+i+"\"]}]}";
            requested = urllib.request.Request(url)
            requested.add_header("X-Naver-Client-Id", client_id)
            requested.add_header("X-Naver-Client-Secret", client_secret)
            requested.add_header("Content-Type", "application/json")
            response = urllib.request.urlopen(requested, data=body.encode("utf-8"))
            rescode = response.getcode()

            if(rescode==200):
                response_body = response.read()
                output_data = response_body.decode('utf-8')
            else:
                print('Error code:'+ rescode)
                pass

            result = json.loads(output_data)


            date = [a['period'] for a in result['results'][0]['data']]


            aa = pd.DataFrame({'date':date, 
                          i:[a['ratio'] for a in result['results'][0]['data']],
                          })

            # 일일 데이터 계산 


            sleep(0.5)
            total = aa[i].sum()


            aa[i] = aa[i].apply(lambda x :((x / total)*float(a)))

        else : 
            pass

    except (TypeError, IndexError,KeyError,ValueError):
        print(" 타입 or 인덱스 에러,Value 에러:",i)
        error.append(i)
        pass


    except HTTPError:
        print('http 에러:', i )
        error.append(i)
        pass

        #여기부터 새로코딩함 
    try: 
        if type(search_keyword(i)) !=str : 
            searchword = i.replace(" ","")
            a = search_keyword(searchword)        

            dt_index = pd.date_range(start=time_month, end= yesterday)
            dt_list = dt_index.strftime("%Y-%m-%d").tolist()

            date = pd.DataFrame(data=dt_list, columns=['날짜'])

            spred = pd.merge(date,aa,left_on='날짜',right_on='date',how='outer')
            spred.drop(['date'],inplace=True, axis=1)

            spred.replace(np.nan,0,inplace=True)


            a=[] 

            for j in range(len(spred)) : 
                a.append(today)


            spred['수집날짜'] = a

            spred = spred[['날짜','수집날짜',i]]

            if i in dictionary_yester:

                pass

            else : 
                dictionary_yester[i]=[]



            dictionary_now[i]=spred

        else:
            pass

    except (TypeError, IndexError,KeyError,ValueError):

        pass

    sleep(0.5)
    try :
        if type(search_keyword(i)) !=str : 
            searchword = i.replace(" ","")
            a = search_keyword(searchword)        

            if dictionary_yester[i]==[]:
                dictionary_yester[i] = dictionary_now[i]
               # for k in range(3):
                #    sleep(1)
                 #   gc2.append_row(dictionary_now[i].iloc[-1-k,:].values.tolist()+[i])


            else : 
                pass 
        else:
            pass


    except (TypeError, IndexError,KeyError):
          pass


    except ValueError:


            yester_df= dictionary_yester[i]
            today_df = dictionary_now[i]


            yester_df.set_index('날짜',inplace=True)

            today_df.set_index('날짜',inplace=True)

            yester_df.update(today_df)


            tmpt = today_df.iloc[-1,:]

            yester_df = yester_df.append(tmpt)

            yester_df.reset_index(inplace=True)
            today_df.reset_index(inplace=True)

            dictionary_yester[i] = yester_df


    except :
        print('API 에러',i)
        error.append(i)

    sleep(3)

  0%|          | 0/1 [00:00<?, ?it/s]

In [27]:
spark = dictionary_now['스파크플러스']

In [37]:
spark_sum = spark['스파크플러스'].tail(7).sum()

In [41]:
we = dictionary_now['위워크']

In [42]:
we_sum = we['위워크'].tail(7).sum()

In [6]:
!git add .

In [11]:
!git commit -m naver '네이버 일일검색량 계산해서 구글시트에 넣기'

error: pathspec '네이버 일일검색량 계산해서 구글시트에 넣기' did not match any file(s) known to git


In [5]:
!git init

/Users/moo/Desktop/jupyter/git/.git/ 안의 기존 깃 저장소를 다시 초기화했습니다


In [8]:
!git status

현재 브랜치 main
커밋할 변경 사항:
  (use "git restore --staged <file>..." to unstage)
	수정함:        aaaa.ipynb
	새 파일:       "naver/.ipynb_checkpoints/\353\204\244\354\235\264\353\262\204 \354\235\274\354\235\274\352\262\200\354\203\211\353\237\211 \352\263\204\354\202\260\352\270\260-checkpoint.ipynb"
	새 파일:       "naver/\353\204\244\354\235\264\353\262\204 \354\235\274\354\235\274\352\262\200\354\203\211\353\237\211 \352\263\204\354\202\260\352\270\260.ipynb"



In [10]:
!git push -u https://github.com/pepetuna/portfolio.git main

오브젝트 나열하는 중: 9, 완료.
오브젝트 개수 세는 중: 100% (9/9), 완료.
Delta compression using up to 8 threads
오브젝트 압축하는 중: 100% (7/7), 완료.
오브젝트 쓰는 중: 100% (7/7), 7.12 KiB | 7.12 MiB/s, 완료.
Total 7 (delta 1), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (1/1), done.
To https://github.com/pepetuna/portfolio.git
   6db86f2..e7ead77  main -> main
branch 'main' set up to track 'https://github.com/pepetuna/portfolio.git/main'.
